# Next-Day Factor in Time Series Forecasting

'Next-Day Factor' refers to the prediction of numerical changes at a future point in time based on certain characteristics in historical data in time series forecasting. In this context, the Next-Day Factor predicts the numerical changes at the next time point (i.e., the following day) based on certain features in historical data (such as the day of the week, date, etc.).


**Steps**：

1. **Data Filtering and Processing:**：

   - Filter data starting from "2014-03-01" up to a specific "month_index" before. Here, "month_index" represents a specified month index used to select data for a specific period.
   **Result**： The DataFrame only retains data from "2014-03-01" up to the specified month.

2. **Adding Timestamps**：

   - Adding various time features, including day of the week, date, week number, and month. 

   **Result**： The dataframe includes columns:"weekday"(day of the week), "day"（date）、"week"（week number）and "month"(month). 

3. **Calculate the Next-Day Factor**：

   - Calculate the average purchase amount and redemption amount for each weekday, and divided these averages by the overall average of purchases and redemptions to obtain the Next-Day Factor. 

   **Result**： `mean_of_each_weekday` DataFrame contains the average purchase and redemption amounts for each weekday, along with the calculated purchase and redemption Next-Day Factors. 

4. **Merge Next-Day Factor into the original dataset**：


   **Result**: The original dataset has been added with columns for the purchase and redemption Next-Day Factors. 

5. **Counting the frequency of Next-Day Occurences on Dates (1-31)**：

   - Count the frequency of each date occuring on each weekday, along with the corresponding Next-Dat Factor

   **Result**： `weekday_count`dataframe contains the frequency of each date occuring on weekdays, and the purchase and redemption Next-Day Factors calculated based on frequency of weekdays. 

6. **Weighting the Next-Day Factor Based on Frequency to Obtain the Date  Factor**：

   **Result**： `day_rate` data frame includes the purchase and redemption Date Factors for each date。

7. **Obtaining the Base by Excluding the Mean Date Residual from the Training Set**：

   - Calculated the mean purchase and redemption for each date, then merged the weighted Date Factor with the mean to obtain the adjusted base value. 

   **Results**： `day_base`data set contains the base purchase and redemption values for each date

8. **Generating Test Set Data**：

   - Generated test set data based on month, date, and base values. 

   **Result**： The generated test set data based on month, date, and base values

9. **Obtaining the Final Prediction results based on the Base and Next-Day Factor**：

   - Merged the test set data with the Next-Day Factor, recalculated the purchase and redemption amounts. 

   **Results**： `day_base` dataframe was recalculated, with the purchase and redemption amounts adjusted based on the purchase and redemption Next-Day Factors, to obtain the final prediction result.

## Project Explanation
Based on the above, the approach adopted by this project is to use methods that extract cyclical features from time series for forecasting:
1. Calculate cyclical factors (factors)
2. Calculate the base
3. Prediction = base * factors

Predicting the future from the past is a classic time series problem. Data such as payment data, passenger flow data, and traffic data exhibit obvious cyclical patterns. From a forecasting perspective, cyclicity is key; capturing the cyclical nature effectively completes a significant part of the task.

Reminder:The project's task is to predict the user purchase and redemption amounts from September 1 to September 30, 2014, based on the user subscription and redemption data table from July 1, 2013, to August 31, 2014, as well as provided data on interest rates and user information.

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns


## Data preparing

In [2]:
# Load dataset
def load_data(path: str='user_balance_table.csv'):
    data_balance=pd.read_csv(path)
    data_balance['date'] = pd.to_datetime(data_balance['report_date'], format= "%Y%m%d")
    return data_balance.reset_index(drop=True)

In [3]:
# Add timestamp
def add_timestamp(data: pd.DataFrame):
    data_balance = data.copy()
    data_balance['day'] = data_balance['date'].dt.day
    data_balance['month'] = data_balance['date'].dt.month
    data_balance['year'] = data_balance['date'].dt.year
    data_balance['week'] = data_balance['date'].dt.week
    data_balance['weekday'] = data_balance['date'].dt.weekday
    return data_balance.reset_index(drop=True)


In [4]:
# Get sum
def get_total_balance(data: pd.DataFrame):
    temp_data = data.copy()
    temp_data = temp_data.groupby(['date'])['total_purchase_amt','total_redeem_amt'].sum()
    temp_data.reset_index(inplace=True)
    return temp_data.reset_index(drop=True)

In [5]:
# Generate test set
def generate_test_data(data: pd.DataFrame)->pd.DataFrame:
    total_balance = data.copy()
    start = datetime.datetime(2014,9,1)
    testdata = []
    while start != datetime.datetime(2014,10,15):
        temp = [start, np.nan, np.nan]
        testdata.append(temp)
        start += datetime.timedelta(days = 1)
    testdata = pd.DataFrame(testdata)
    testdata.columns = total_balance.columns

    total_balance = pd.concat([total_balance, testdata], axis = 0)
    total_balance = total_balance.reset_index(drop=True)
    return total_balance.reset_index(drop=True)

In [6]:
balance_data = load_data('/Users/yupengshen/DA_portfolio/cashflow/data/user_balance_table.csv')

In [7]:
balance_data = add_timestamp(balance_data)

/var/folders/rq/rtxryn3j4c91cgyhb885lxn40000gn/T/ipykernel_9245/3350547994.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data_balance['week'] = data_balance['date'].dt.week


In [8]:
total_balance = get_total_balance(balance_data)

/var/folders/rq/rtxryn3j4c91cgyhb885lxn40000gn/T/ipykernel_9245/1560135031.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp_data = temp_data.groupby(['date'])['total_purchase_amt','total_redeem_amt'].sum()


In [9]:
total_balance = generate_test_data(total_balance)

In [10]:
total_balance = add_timestamp(total_balance)

/var/folders/rq/rtxryn3j4c91cgyhb885lxn40000gn/T/ipykernel_9245/3350547994.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data_balance['week'] = data_balance['date'].dt.week


In [11]:
total_balance.tail()

,date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday
466,2014-10-10,NaN,NaN,10,10,2014,41,4
467,2014-10-11,NaN,NaN,11,10,2014,41,5
468,2014-10-12,NaN,NaN,12,10,2014,41,6
469,2014-10-13,NaN,NaN,13,10,2014,42,0
470,2014-10-14,NaN,NaN,14,10,2014,42,1


In [12]:
def generate_base(df:pd.DataFrame, month_index:int) -> pd.DataFrame:
    # Chose the time range
    total_balance=df.copy()
    total_balance = total_balance[['date', 'total_purchase_amt', 'total_redeem_amt']]
    total_balance = total_balance[(total_balance['date'] >= pd.to_datetime('2014/3/1')) & (total_balance['date'] < pd.to_datetime('2014/{}/1'.format(month_index)))]
    
    # Add timestamp
    total_balance['weekday'] = total_balance['date'].dt.weekday  
    total_balance['day'] = total_balance['date'].dt.day          
    total_balance['week'] = total_balance['date'].dt.week        
    total_balance['month'] = total_balance['date'].dt.month 
    
    # Calculate the Next-Day Factor
    mean_of_each_weekday = total_balance[['weekday', 'total_purchase_amt', 'total_redeem_amt']].groupby('weekday', as_index=False).mean()
    for name in ['total_purchase_amt', 'total_redeem_amt']:
        mean_of_each_weekday = mean_of_each_weekday.rename(columns={name: name+'_next_day_factor'})
    mean_of_each_weekday['total_purchase_amt_next_day_factor'] /= np.mean(total_balance['total_purchase_amt'])
    mean_of_each_weekday['total_redeem_amt_next_day_factor'] /= np.mean(total_balance['total_redeem_amt'])
    
    # Merge 'mean_of_each_weekday' with 'total_balance'ArithmeticError
    total_balance = pd.merge(total_balance, mean_of_each_weekday, on='weekday', how='left')
    
    # Count the frequency of day occuring on dates from 1st to 31st
    weekday_count = total_balance[['day', 'weekday', 'date']].groupby(['day', 'weekday'], as_index=False).count()
    weekday_count = pd.merge(weekday_count, mean_of_each_weekday, on='weekday')
    
    # Weight the next-day factor based on frequency to obtain the Date factor
    weekday_count['total_purchase_amt_next_day_factor'] *= weekday_count['date'] / len(np.unique(total_balance['month']))  # Calculate the date factor for purchase amounts by weighting according to frequency
    weekday_count['total_redeem_amt_next_day_factor'] *= weekday_count['date'] / len(np.unique(total_balance['month']))    # Calculate the date factor for redemption amounts by weighting according to frequency
    day_rate = weekday_count.drop(['weekday', 'date'], axis=1).groupby('day', as_index=False).sum()  
    
    day_rate = day_rate.rename(columns={
    'total_purchase_amt_next_day_factor': 'total_purchase_amt_date_factor',
    'total_redeem_amt_next_day_factor': 'total_redeem_amt_date_factor'})
    
    
    # Subtract the date residuals from the average of all date to obtain the base
    day_mean = total_balance[['day'] + ['total_purchase_amt', 'total_redeem_amt']].groupby('day', as_index=False).mean() 
    day_base = pd.merge(day_mean, day_rate, on='day', how='left')
    day_base['total_purchase_amt'] /= day_base['total_purchase_amt_date_factor'] # Divide the purchase amount by the next-day purchase factor
    day_base['total_redeem_amt'] /= day_base['total_redeem_amt_date_factor'] 
    
    # Generate test set
    for index, row in day_base.iterrows():
        if month_index in (2, 4, 6, 9) and row['day'] == 31:
            break
        day_base.loc[index, 'date'] = datetime.datetime(2014, month_index, int(row['day']), 0, 0, 0)
        
    # Obtain the final forecast results based on the base and the date factors
    day_base['weekday'] = day_base['date'].dt.weekday 
    day_base = day_base[['date', 'weekday'] + ['total_purchase_amt', 'total_redeem_amt']]  
    day_base = pd.merge(day_base, mean_of_each_weekday, on='weekday') 
    day_base['total_purchase_amt'] *= day_base['total_purchase_amt_next_day_factor'] 
    day_base['total_redeem_amt'] *= day_base['total_redeem_amt_next_day_factor']      
    
    day_base = day_base.sort_values('date')[['date'] + ['total_purchase_amt', 'total_redeem_amt']]
    return day_base

In [13]:
base_list = []

for i in range(4, 10):
    base_list.append(generate_base(total_balance, i).reset_index(drop=True))

base = pd.concat(base_list).reset_index(drop=True)
for i in ['total_purchase_amt','total_redeem_amt']:
    base = base.rename(columns={i: i+'_base'})

data = pd.merge(total_balance.reset_index(drop=True), base.reset_index(drop=True), on='date', how='left').reset_index(drop=True)

data['purchase_residual'] = data['total_purchase_amt'] / data['total_purchase_amt_base']

data['redeem_residual'] = data['total_redeem_amt'] / data['total_redeem_amt_base']

/var/folders/rq/rtxryn3j4c91cgyhb885lxn40000gn/T/ipykernel_9245/1659467189.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  total_balance['week'] = total_balance['date'].dt.week
/var/folders/rq/rtxryn3j4c91cgyhb885lxn40000gn/T/ipykernel_9245/1659467189.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  total_balance['week'] = total_balance['date'].dt.week
/var/folders/rq/rtxryn3j4c91cgyhb885lxn40000gn/T/ipykernel_9245/1659467189.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  total_balance['week'] = total_balance['date'].dt.week
/var/folders/rq/rtxryn3j4c91cgyhb885lxn40000gn/T/ipykernel_9245/1659467189.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.


In [14]:
data.head()

,date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday,total_purchase_amt_base,total_redeem_amt_base,purchase_residual,redeem_residual
0,2013-07-01,32488348.0,5525022.0,1,7,2013,27,0,NaN,NaN,NaN,NaN
1,2013-07-02,29037390.0,2554548.0,2,7,2013,27,1,NaN,NaN,NaN,NaN
2,2013-07-03,27270770.0,5953867.0,3,7,2013,27,2,NaN,NaN,NaN,NaN
3,2013-07-04,18321185.0,6410729.0,4,7,2013,27,3,NaN,NaN,NaN,NaN
4,2013-07-05,11648749.0,2763587.0,5,7,2013,27,4,NaN,NaN,NaN,NaN


In [15]:
data[data['date'] == pd.Timestamp('2014-03-31')]

,date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday,total_purchase_amt_base,total_redeem_amt_base,purchase_residual,redeem_residual
273,2014-03-31,398884905.0,423852634.0,31,3,2014,14,0,NaN,NaN,NaN,NaN


In [16]:
# Restructure the result table
data = data[['date','total_purchase_amt','total_purchase_amt_base','purchase_residual','total_redeem_amt','total_redeem_amt_base','redeem_residual']]

baseline_results = data[data['date'] >= pd.Timestamp('2014-04-01')]


In [17]:
# We start predicting the baseline from 2014-04-01
baseline_results.head()


,date,total_purchase_amt,total_purchase_amt_base,purchase_residual,total_redeem_amt,total_redeem_amt_base,redeem_residual
274,2014-04-01,453320585.0,6.207296e+08,0.730303,277429358.0,4.373304e+08,0.634370
275,2014-04-02,355347118.0,4.342629e+08,0.818276,272612066.0,3.578884e+08,0.761724
276,2014-04-03,363877120.0,4.734257e+08,0.768604,266605457.0,3.791019e+08,0.703255
277,2014-04-04,251895894.0,3.703544e+08,0.680148,200192637.0,2.161569e+08,0.926145
278,2014-04-05,202336542.0,3.093513e+08,0.654067,163199682.0,9.861873e+07,1.654855


In [18]:
# We can see  our baseline prediction is for 2014 September
baseline_results[baseline_results['date']>=('2014-09-01')]

,date,total_purchase_amt,total_purchase_amt_base,purchase_residual,total_redeem_amt,total_redeem_amt_base,redeem_residual
427,2014-09-01,NaN,3.933503e+08,NaN,NaN,3.066653e+08,NaN
428,2014-09-02,NaN,3.085214e+08,NaN,NaN,2.570847e+08,NaN
429,2014-09-03,NaN,3.430706e+08,NaN,NaN,3.486316e+08,NaN
430,2014-09-04,NaN,3.555449e+08,NaN,NaN,2.860506e+08,NaN
431,2014-09-05,NaN,2.909838e+08,NaN,NaN,2.508833e+08,NaN
432,2014-09-06,NaN,2.184999e+08,NaN,NaN,1.688910e+08,NaN
433,2014-09-07,NaN,1.928032e+08,NaN,NaN,1.710349e+08,NaN
434,2014-09-08,NaN,3.528802e+08,NaN,NaN,3.233303e+08,NaN
435,2014-09-09,NaN,3.551925e+08,NaN,NaN,2.869784e+08,NaN
436,2014-09-10,NaN,3.947967e+08,NaN,NaN,3.046577e+08,NaN


In [19]:
df=baseline_results[(baseline_results['date']>='2014-09-01')& (baseline_results['date']<='2014-09-30')][['date','total_purchase_amt_base','total_redeem_amt_base']]

In [20]:
df

,date,total_purchase_amt_base,total_redeem_amt_base
427,2014-09-01,3.933503e+08,3.066653e+08
428,2014-09-02,3.085214e+08,2.570847e+08
429,2014-09-03,3.430706e+08,3.486316e+08
430,2014-09-04,3.555449e+08,2.860506e+08
431,2014-09-05,2.909838e+08,2.508833e+08
432,2014-09-06,2.184999e+08,1.688910e+08
433,2014-09-07,1.928032e+08,1.710349e+08
434,2014-09-08,3.528802e+08,3.233303e+08
435,2014-09-09,3.551925e+08,2.869784e+08
436,2014-09-10,3.947967e+08,3.046577e+08


In [27]:
df.to_csv('/Users/yupengshen/DA_portfolio/cashflow/baseline_result.csv', index=False) 